In [1]:
# # TensorFlow and tf.keras
import tensorflow as tf
import pandas as pd

#if working with tensorflow > v2 uncomment all of the following
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import utils


#if working with tf < v2 comment above and uncomment next block of code

# import keras
# import keras.backend
# from keras import layers
# from keras import models
# import keras.utils

print(keras.__version__)
print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import time
import h5py
import sklearn
from sklearn.utils import shuffle

import os

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

2.2.4-tf
1.13.1


Import data from RNN_data.npz

In [2]:
# training data
traindata_sig = np.load('data/dummyvectors_signal.npz')
traindata_bkg = np.load('data/dummyvectors_bkg.npz')
trainlabels = np.concatenate((traindata_sig['labels'],traindata_bkg['labels']),axis=0)
trainvectors = np.concatenate((traindata_sig['fourvectors'], traindata_bkg['fourvectors']), axis = 1)
trainradii = np.concatenate((traindata_sig['radiilab'], traindata_bkg['radiilab']), axis = 0)
traintheta = np.concatenate((traindata_sig['thetalab'], traindata_bkg['thetalab']), axis = 0)
trainz = np.concatenate((traindata_sig['zlab'], traindata_bkg['zlab']), axis = 0)
n_train = len(trainlabels)
trainlabels = trainlabels.reshape(int(n_train/20), 20, 2)
trainvectors = trainvectors.reshape(int(n_train/20), 20, 4)

# testing data
testdata_sig = np.load('data/dummyvectors_sgn_test.npz')
testdata_bkg = np.load('data/dummyvectors_bkg_test.npz')
testlabels = np.concatenate((testdata_sig['labels'], testdata_bkg['labels']),axis=0)
testvectors = np.concatenate((testdata_sig['fourvectors'], testdata_bkg['fourvectors']), axis = 1)
testradii = np.concatenate((testdata_sig['radiilab'], testdata_bkg['radiilab']), axis = 0)
test_theta = np.concatenate((testdata_sig['thetalab'], testdata_bkg['thetalab']), axis = 0)
testz = np.concatenate((testdata_sig['zlab'], testdata_bkg['zlab']), axis = 0)
n_test = len(testlabels)
testlabels = testlabels.reshape(int(n_train/20), 20, 2)
testvectors = testvectors.reshape(int(n_train/20), 20, 4)

In [3]:
def build_model(rnn_units, batch, n_train):
    input1 = layers.Input(name = 'vector_input', batch_shape=(None,batch, 4) )
    x = layers.GRU(8*rnn_units, return_sequences = True, name = 'gru1')(input1)
    x = layers.GRU(4*rnn_units, return_sequences = True, name = 'gru2')(x)
#     input2 = layers.Input(shape=(1,), name = 'z_input')
#     input3 = layers.Input(shape=(2,), name = 'radii_input')
#     input4 = layers.Input(shape=(1,), name = 'theta_input')
#     x = layers.concatenate(inputs = [xf, input2, input3, input4],axis=-1, name = 'concat')
    output = layers.Dense(2, activation='softmax', name = 'softmax')(x)
    model = models.Model(inputs=input1, outputs=output)
    optimizer = keras.optimizers.Adam(clipnorm = 1.0)
    model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['categorical_crossentropy', 'accuracy'])
    return model

In [4]:
rnn_units = 256
batch = 20
RNN = build_model(rnn_units, batch, 10)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
example_batch = trainvectors[:10,:,:]
example_labels = trainlabels[:10,:,:]
example_radii = trainradii[:10]
example_theta = traintheta[:10]
example_z = trainz[:10]
print(example_radii.shape)
example_result = RNN.predict(x = [example_batch])
results = RNN.evaluate(x = [example_batch], y = example_labels)
print(example_result)
print(results)

(10, 2)
10/10 [==============================] - 0s 42ms/sample - loss: 0.6139 - categorical_crossentropy: 0.6139 - acc: 0.7300
[[[0.5013546  0.49864542]
  [0.58665097 0.41334903]
  [0.6075327  0.3924673 ]
  [0.6895189  0.3104811 ]
  [0.5611242  0.43887585]
  [0.50480497 0.49519503]
  [0.4379451  0.56205493]
  [0.50446326 0.49553677]
  [0.5268246  0.47317544]
  [0.51537615 0.48462382]
  [0.47754103 0.52245903]
  [0.4846629  0.5153371 ]
  [0.5322693  0.46773067]
  [0.5311696  0.4688304 ]
  [0.5325233  0.46747676]
  [0.53497106 0.46502897]
  [0.53103536 0.46896464]
  [0.54738414 0.45261586]
  [0.47598034 0.52401966]
  [0.5105659  0.48943412]]

 [[0.49558976 0.5044102 ]
  [0.6102272  0.38977277]
  [0.70345986 0.29654017]
  [0.63801533 0.36198467]
  [0.5051426  0.49485737]
  [0.54658455 0.45341545]
  [0.4947838  0.50521624]
  [0.49582866 0.5041714 ]
  [0.34557077 0.65442926]
  [0.34564745 0.65435255]
  [0.44708973 0.55291027]
  [0.5133408  0.48665923]
  [0.44493112 0.5550689 ]
  [0.5674176

In [6]:
RNN = build_model(rnn_units, batch, n_train)

In [7]:
checkpoint_path = "model/RNN_4GRU_gradclip.h5"
if not os.path.exists("model"):
    os.mkdir("model")

# Create checkpoint callback
# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
#                                                  save_best_only=True,
#                                                  verbose=1)
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', 
                                   verbose=1, save_best_only=True, 
                                   save_weights_only=False, mode='auto')    
EPOCHS = 1000
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = RNN.fit(
  trainvectors, trainlabels,
  epochs=EPOCHS, validation_split = 0.2, verbose = 0,
  callbacks=[early_stop, model_checkpoint])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


AlreadyExistsError: Resource __per_step_19/training/Adam/gradients/gru2_1/while/ReadVariableOp/Enter_grad/ArithmeticOptimizer/AddOpsRewrite_Add/tmp_var/N10tensorflow19TemporaryVariableOp6TmpVarE
	 [[{{node training/Adam/gradients/gru2_1/while/ReadVariableOp/Enter_grad/ArithmeticOptimizer/AddOpsRewrite_Add/tmp_var}}]]

In [ ]:
# def plot_history(history):
#     hist = pd.DataFrame(history.history)
#     print(hist.keys())
# #   hist['epoch'] = history.epoch
# #   plt.figure()
# #   plt.xlabel('Epoch')
# #   plt.ylabel('Mean Square Error')
# #   plt.plot(hist['epoch'], hist['mean_squared_error'],
# #            label='Train Error')
# #   plt.plot(hist['epoch'], hist['val_mean_squared_error'],
# #            label = 'Val Error')
# #   plt.ylim([0,100])
# #   plt.legend()
# #   plt.show()

# plot_history(history)

In [ ]:
!ls model
best_model = keras.models.load_model('model/RNN.h5')
best_model.summary()
results = best_model.evaluate(testvectors, testlabels, verbose = 0)
print(results)

In [ ]:
# Plot ROC
predict = best_model.predict(trainvectors)
print("Ex. signal prediction:")
print(predict[0:,:,:])
print("Ex. background prediction:")
print(predict[-10:-1,:,:])
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(trainlabels[:,:,0].flatten(), predict[:,:,0].flatten())
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='blue', label='auc = %.3f' % (roc_auc))
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k', label='random chance')
plt.xlim([0, 1.0])
plt.ylim([0, 1.0])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.title('receiver operating curve')
plt.legend(loc="lower right")
plt.show()